In [1]:
import itertools
import numpy as np
import pandas as pd

In [2]:
cancer = (
    pd.read_csv("../data/BreastCancer.csv", header=0)
    .sample(frac=1)
    .dropna(axis=0)
    .drop(columns=["id"])
    .replace({"diagnosis": {"M": 1, "B": 0}})
)
display(cancer)

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave_points_mean,symmetry_mean,fractal_dimension_mean
235,0,14.030,21.25,89.79,603.4,0.09070,0.06945,0.01462,0.01896,0.1517,0.05835
443,0,10.570,18.32,66.82,340.9,0.08142,0.04462,0.01993,0.01111,0.2372,0.05768
402,0,12.960,18.29,84.18,525.2,0.07351,0.07899,0.04057,0.01883,0.1874,0.05899
428,0,11.130,16.62,70.47,381.1,0.08151,0.03834,0.01369,0.01370,0.1511,0.06148
247,0,12.890,14.11,84.95,512.2,0.08760,0.13460,0.13740,0.03980,0.1596,0.06409
...,...,...,...,...,...,...,...,...,...,...,...
292,0,12.950,16.02,83.14,513.7,0.10050,0.07943,0.06155,0.03370,0.1730,0.06470
86,1,14.480,21.46,94.25,648.2,0.09444,0.09947,0.12040,0.04938,0.2075,0.05636
116,0,8.950,15.76,58.74,245.2,0.09462,0.12430,0.09263,0.02308,0.1305,0.07163
539,0,7.691,25.44,48.34,170.4,0.08668,0.11990,0.09252,0.01364,0.2037,0.07751


In [3]:
display(cancer.dtypes)

diagnosis                   int64
radius_mean               float64
texture_mean              float64
perimeter_mean            float64
area_mean                 float64
smoothness_mean           float64
compactness_mean          float64
concavity_mean            float64
concave_points_mean       float64
symmetry_mean             float64
fractal_dimension_mean    float64
dtype: object

In [9]:
def calculate_entropy(y):
    if isinstance(y, pd.Series):
        p = y.value_counts() / len(y)
        entropy = np.sum(-p * np.log2(p + 1e-9))
        return entropy
    else:
        raise ValueError("wrong data type")


def calculate_information_gain(y, m, func=calculate_entropy): # m = mask
    a = sum(m)
    b = m.shape[0] - a
  
    if a == 0 or b == 0: 
        ig = 0
    else:
        ig = func(y) - (a / (a + b)) * func(y[m]) - (b / (a + b)) * func(y[-m])

    return ig


def categorical_options(z):
    z = np.unique(z)

    out = []
    for l in range(len(z) + 1):
        for s in itertools.combinations(z, l):
            out.append(list(s))

    return out[1:-1]


def max_information_gain_split(x, y, func=calculate_entropy):
    splitval = []
    ig = [] 

    numeric_variable = True if x.dtypes != 'O' else False

    if numeric_variable:
        options = x.sort_values().unique()[1:]
    else: 
        options = categorical_options(x)

    for val in options:
        mask = x < val if numeric_variable else x.isin(val)
        val_ig = calculate_information_gain(y, mask, func)
        splitval.append(val)
        ig.append(val_ig)
        
    if len(ig) == 0:
        out = (None, None, None, False)
    else:
        best_ig = max(ig)
        best_ig_index = ig.index(best_ig)
        best_split = splitval[best_ig_index]
        out = (best_ig, best_split, numeric_variable, True)
    
    return out


def get_best_split(y, data):
    masks = data.drop(y, axis= 1).apply(max_information_gain_split, y = data[y])
    if sum(masks.loc[3,:]) == 0:
        return(None, None, None, None)
    else:
        masks = masks.loc[:, masks.loc[3, :]]

        split_variable = max(masks)
        split_value = masks[split_variable][1] 
        split_ig = masks[split_variable][0]
        split_numeric = masks[split_variable][2]

        return(split_variable, split_value, split_ig, split_numeric)


def make_split(variable, value, data, is_numeric):
    if is_numeric:
        data_1 = data[data[variable] < value]
        data_2 = data[(data[variable] < value) == False]
    else:
        data_1 = data[data[variable].isin(value)]
        data_2 = data[(data[variable].isin(value)) == False]
    return(data_1, data_2)


def make_prediction(data, target_factor):
    if target_factor:
        pred = data.value_counts().idxmax()
    else:
        pred = data.mean()
    return pred


def train_tree(
    data, 
    y, 
    target_factor, 
    max_depth=None, 
    min_samples_split=None, 
    min_information_gain=1e-20, 
    counter=0, 
    max_categories=20
):
    if counter == 0:
        types = data.dtypes
        check_columns = types[types == "object"].index
        for column in check_columns:
            var_length = len(data[column].value_counts()) 
            if var_length > max_categories:
                raise ValueError(
                    'The variable ' + column + ' has '+ str(var_length) + 
                    ' unique values, which is more than the accepted ones: ' + 
                    str(max_categories)
                )

    # Check for depth conditions
    if max_depth == None:
        depth_cond = True
    else:
        if counter < max_depth:
            depth_cond = True
        else:
            depth_cond = False

    # Check for sample conditions
    if min_samples_split == None:
        sample_cond = True
    else:
        if data.shape[0] > min_samples_split:
            sample_cond = True
        else:
            sample_cond = False

    # Check for ig condition
    if depth_cond & sample_cond:
        var, val, ig, var_type = get_best_split(y, data)
        
        if ig is not None and ig >= min_information_gain:
            counter += 1

            left, right = make_split(var, val, data, var_type)

            split_type = "<=" if var_type else "in"
            question =   "{} {} {}".format(var, split_type, val)
            subtree = {question: []}

            yes_answer = train_tree(
                left,
                y, 
                target_factor, 
                max_depth, 
                min_samples_split, 
                min_information_gain, 
                counter
            )

            no_answer = train_tree(
                right, 
                y, 
                target_factor, 
                max_depth, 
                min_samples_split, 
                min_information_gain, 
                counter
            )

            if yes_answer == no_answer:
                subtree = yes_answer
            else:
                subtree[question].append(yes_answer)
                subtree[question].append(no_answer)
        else:
            pred = make_prediction(data[y], target_factor)
            return pred
    else:
        pred = make_prediction(data[y], target_factor)
        return pred

    return subtree

max_depth = 3
min_samples_split = 10
min_information_gain  = 1e-5

decisiones = train_tree(
    data=cancer, 
    y='diagnosis', 
    target_factor=True, 
    max_depth=max_depth, 
    min_samples_split=min_samples_split, 
    min_information_gain=min_information_gain
)
decisiones

{'texture_mean <= 18.47': [{'texture_mean <= 15.05': [0,
    {'texture_mean <= 15.11': [1, 0]}]},
  {'texture_mean <= 20.2': [0, {'texture_mean <= 26.99': [1, 0]}]}]}

In [13]:
def clasificar_datos(observacion, arbol):
    question = list(arbol.keys())[0] 

    if question.split()[1] == '<=':
        if observacion[question.split()[0]] <= float(question.split()[2]):
            answer = arbol[question][0]
        else:
            answer = arbol[question][1]
    else:
        if observacion[question.split()[0]] in (question.split()[2]):
            answer = arbol[question][0]
        else:
            answer = arbol[question][1]

    if not isinstance(answer, dict):
        return answer
    else:
        residual_tree = answer
        return clasificar_datos(observacion, answer)
    
yfit = []
for i in range(len(cancer)):
    yfit.append(clasificar_datos(cancer.iloc[i, :], decisiones))

In [16]:
print(cancer.diagnosis.values[-10:])
print(yfit[-10:])

[0 1 0 1 0 0 1 0 0 0]
[0, 0, 1, 0, 0, 0, 1, 0, 1, 0]
